In [1]:
from rich import print
import logging

logging.basicConfig(level=logging.INFO)

## Observations from [Phenocam](hhttps://phenocam.nau.edu/webcam/)

### Data from harvard site


In [2]:
from springtime.datasets import Phenocam

Retrieve meta data on "harvard" site.


In [3]:
from springtime.datasets.phenocam import list_sites, list_rois

sites = list_sites()
sites[sites.site == "harvard"]

,site,elev,contact1,contact2,date_start,date_end,nimage,tzoffset,active,infrared,...,MAP_worldclim,dominant_species,primary_veg_type,secondary_veg_type,koeppen_geiger,ecoregion,wwf_biome,landcover_igbp,site_acknowledgements,geometry
226,harvard,340.0,Andrew Richardson <andrew DOT richardson AT na...,Bill Munger <jwmunger AT seas DOT harvard DOT ...,2008-04-04,2024-01-08,190650,-5.0,True,N,...,1139.0,"Quercus rubra, Acer rubrum, Pinus strobus",DB,EN,Dfb,5.0,4.0,5.0,The Harvard EMS site is supported is an AmeriF...,POINT (-72.17150 42.53780)


List rois of harvard


In [4]:
rois = list_rois()
rois[rois.site == "harvard"]

,site,veg_type,roi_id_number,description,first_date,last_date,site_years,missing_data_pct,geometry
333,harvard,DB,1,Deciduous trees in foreground,2008-04-04,2024-01-08,15.7,0.0,POINT (-72.17150 42.53780)
334,harvard,DB,1000,Deciduous trees in foreground,2008-04-04,2024-01-08,15.7,0.0,POINT (-72.17150 42.53780)


In [16]:
# Use $ in site name to get an exact match
dataset = Phenocam(site="harvard$", years=(2010, 2015))
print(dataset)

Phenocam(
    dataset='phenocam',
    years=YearRange(start=2010, end=2015),
    veg_type=None,
    frequency='3',
    variables=(),
    site='harvard$',
    roi_id=None,
    area=None
)

In [6]:
dataset.download()

INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Looking for data
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_0001_3day.csv
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_1000_3day.csv


[PosixPath('/home/peter/.cache/springtime/phenocam/harvard_DB_0001_3day.csv'),
 PosixPath('/home/peter/.cache/springtime/phenocam/harvard_DB_1000_3day.csv')]

In [7]:
df = dataset.raw_load()
df.head()

INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Looking for data
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_0001_3day.csv
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_1000_3day.csv


,site,roi_id_number,veg_type,date,year,doy,image_count,midday_filename,midday_r,midday_g,...,smooth_rcc_90,smooth_ci_gcc_mean,smooth_ci_gcc_50,smooth_ci_gcc_75,smooth_ci_gcc_90,smooth_ci_rcc_mean,smooth_ci_rcc_50,smooth_ci_rcc_75,smooth_ci_rcc_90,int_flag
0,harvard,0001,DB,2008-01-05,2008,5,NaN,NaN,NaN,NaN,...,0.31592,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,1.0
1,harvard,0001,DB,2008-01-06,2008,6,NaN,NaN,NaN,NaN,...,0.31552,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,1.0
2,harvard,0001,DB,2008-01-07,2008,7,NaN,NaN,NaN,NaN,...,0.31514,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,1.0
3,harvard,0001,DB,2008-01-08,2008,8,NaN,NaN,NaN,NaN,...,0.31476,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,1.0
4,harvard,0001,DB,2008-01-09,2008,9,NaN,NaN,NaN,NaN,...,0.31441,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,1.0


In [8]:
df = dataset.load()
df.head()

INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Looking for data
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_0001_3day.csv
INFO:/home/peter/phenology/springtime/src/springtime/datasets/phenocam.py:Found /home/peter/.cache/springtime/phenocam/harvard_DB_1000_3day.csv


,site,roi_id_number,veg_type,date,year,doy,image_count,midday_filename,midday_r,midday_g,...,smooth_rcc_90,smooth_ci_gcc_mean,smooth_ci_gcc_50,smooth_ci_gcc_75,smooth_ci_gcc_90,smooth_ci_rcc_mean,smooth_ci_rcc_50,smooth_ci_rcc_75,smooth_ci_rcc_90,int_flag
727,harvard,0001,DB,2010-01-01,2010,1,NaN,NaN,NaN,NaN,...,0.31163,0.00306,0.00305,0.00318,0.00370,0.00721,0.00795,0.00699,0.00660,NaN
728,harvard,0001,DB,2010-01-02,2010,2,26.0,harvard_2010_01_02_120137.jpg,52.19172,71.35788,...,0.31155,0.00301,0.00300,0.00313,0.00364,0.00709,0.00782,0.00688,0.00650,NaN
729,harvard,0001,DB,2010-01-03,2010,3,NaN,NaN,NaN,NaN,...,0.31160,0.00297,0.00296,0.00308,0.00359,0.00699,0.00771,0.00678,0.00640,NaN
730,harvard,0001,DB,2010-01-04,2010,4,NaN,NaN,NaN,NaN,...,0.31175,0.00295,0.00294,0.00307,0.00357,0.00696,0.00767,0.00674,0.00637,NaN
731,harvard,0001,DB,2010-01-05,2010,5,39.0,harvard_2010_01_05_120139.jpg,54.79733,68.07477,...,0.31197,0.00297,0.00297,0.00309,0.00360,0.00701,0.00773,0.00680,0.00642,NaN


### Data from sites around harvard


In [11]:
harvard = {"name": "harvard", "bbox": [-73, 42, -72, 43]}

dataset = Phenocam(area=harvard, years=[2019, 2020])

print(dataset)

Phenocam(
    dataset='phenocam',
    years=YearRange(start=2019, end=2020),
    veg_type=None,
    frequency='3',
    variables=(),
    site=None,
    roi_id=None,
    area=NamedArea(name='harvard', bbox=BoundingBox(xmin=-73.0, ymin=42.0, xmax=-72.0, ymax=43.0))
)

In [ ]:
# Create a data instance
dataset.download()

R[write to console]: Downloading: bbc1_DB_1000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: bbc2_DB_1000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: harvardbarn_DB_1000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: harvardbarn_EN_1000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: harvardbarn2_DB_1000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: harvardbarn2_DB_2000_3day.csv

R[write to console]: -- Flagging outliers!

R[write to console]: -- Smoothing time series!

R[write to console]: Downloading: harvardbarn2_EN_1000_3day.csv

R[write to 

In [ ]:
# Load downloded dataset as a dataframe
df = dataset.load()
df.head()

,site,roi_id_number,veg_type,date,year,doy,image_count,midday_filename,midday_r,midday_g,...,smooth_ci_gcc_mean,smooth_ci_gcc_50,smooth_ci_gcc_75,smooth_ci_gcc_90,smooth_ci_rcc_mean,smooth_ci_rcc_50,smooth_ci_rcc_75,smooth_ci_rcc_90,int_flag,geometry
0,bbc1,1000,DB,2019-01-01,2019,1,NaN,NaN,NaN,NaN,...,0.00296,0.00310,0.00318,0.00323,0.00934,0.00959,0.00974,0.00942,NaN,POINT (-72.17436 42.53508)
1,bbc1,1000,DB,2019-01-02,2019,2,47.0,bbc1_2019_01_02_120002.jpg,84.86659,85.59192,...,0.00287,0.00301,0.00309,0.00313,0.00906,0.00930,0.00944,0.00913,NaN,POINT (-72.17436 42.53508)
2,bbc1,1000,DB,2019-01-03,2019,3,NaN,NaN,NaN,NaN,...,0.00288,0.00302,0.00310,0.00314,0.00911,0.00935,0.00949,0.00918,NaN,POINT (-72.17436 42.53508)
3,bbc1,1000,DB,2019-01-04,2019,4,NaN,NaN,NaN,NaN,...,0.00300,0.00314,0.00322,0.00327,0.00946,0.00971,0.00986,0.00954,NaN,POINT (-72.17436 42.53508)
4,bbc1,1000,DB,2019-01-05,2019,5,81.0,bbc1_2019_01_05_120004.jpg,60.30106,67.32647,...,0.00308,0.00322,0.00331,0.00336,0.00972,0.00998,0.01013,0.00980,NaN,POINT (-72.17436 42.53508)


## TODO

- Extract relevant variables / infer event
- Pivot dataframe such that it has one row per year/site.
- Make nicer storyline in this notebook.
